In [2]:
import pandas as pd
import numpy as np

In [4]:
nonhouse = pd.read_excel("./EA_Data/pp_nonhouse.xlsx")

1. 인덱스 지정 및 수정
- 고성군1,2 구분
- 계 -> 서울특별시, ...

In [5]:
nonhouse = nonhouse[nonhouse["시군구"] != "세종시"]

In [6]:
#고성군
nonhouse.loc[(nonhouse["구분"] == "강원") & (nonhouse["시군구"]=="고성군"), "시군구"] = "고성군1"
nonhouse.loc[(nonhouse["구분"] == "경남") & (nonhouse["시군구"]=="고성군"), "시군구"] = "고성군2"

In [38]:
do = { 
    "경기": "경기도", 
    "강원": "강원도",
    "충북": "충청북도",
    "충남": "충청남도",
    "전북": "전라북도",
    "전남": "전라남도",
    "경북": "경상북도",
    "경남": "경상남도",
    "세종": "세종특별자치시",
    "제주": "제주특별자치도"}

nonhouse["시군구"] = nonhouse.apply(
    lambda row: do.get(row["구분"], row["시군구"]) if row["시군구"] == "계" else row["시군구"],
    axis=1
)
mapping = {
    "서울": "서울특별시",
    "부산": "부산광역시",
    "대구": "대구광역시",
    "인천": "인천광역시",
    "광주": "광주광역시",
    "대전": "대전광역시",
    "울산": "울산광역시",
}
# `구분` 열을 기준으로 `시군구` 값 매핑
nonhouse["시군구"] = nonhouse["구분"].map(mapping).fillna(nonhouse["시군구"])


In [20]:
result = pd.read_excel("./results/ecm_result1.9.xlsx")

In [39]:
nonhouse.reset_index(inplace=True)

In [40]:
# nonhouse.drop(columns=["level_0","index"])
nonhouse.iloc[181]

level_0        181
index          181
월          2009-12
구분              경기
시군구            경기도
미분양         19,325
Name: 181, dtype: object

연도별 개수 체크

In [43]:
before = 2008
year_gap =[0]

for i in range(len(nonhouse)):
    if int(nonhouse.loc[i,"월"][:4]) != before:
        year_gap.append(nonhouse.iloc[i,:].name)
        before += 1
        
year_gap.append(len(nonhouse))
#year_gap

In [44]:
print(year_gap)
print(len(year_gap))


[0, 174, 348, 522, 694, 867, 1040, 1212, 1384, 1556, 1727, 1898, 2069, 2240, 2411, 2582]
16


인덱스 지정

In [45]:
custom_order = list(result.loc[:178,"시군구"])

In [22]:
duplicates = [x for x in set(custom_order) if custom_order.count(x) > 1]
print("중복값:", duplicates) 
    

중복값: []


1. 5개 추가: "통합창원시","통합청주시",("여주시","여주군"),("당진시","당진군"),"세종특별자치시"
2. 180개씩 잘라서 reindex
3. concat으로 합치기

In [46]:
result_form = pd.DataFrame(columns=["월", "구분", "미분양"]) #새로 담을 데이터 프레임

In [47]:
for i, year in zip(range(len(year_gap)), range(2008,2023)):
        
    subsample = nonhouse[year_gap[i]:year_gap[i+1]]

    new_rows = pd.DataFrame([
        {"미분양": 0, "월": year, "구분": "경남", "시군구": "통합창원시"},
        {"미분양": 0, "월": year, "구분": "충남", "시군구": "통합청주시"},
    ])
    if "세종특별자치시" not in subsample["시군구"].to_list():
        print(0)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "세종", "시군구": "세종특별자치시"}])
        ])
    if "마산시" not in subsample["시군구"].to_list():
        print(1)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "경남", "시군구": "마산시"}])
        ])
    if "진해시" not in subsample["시군구"].to_list():
        print(2)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "경남", "시군구": "진해시"}])
        ])
    if "연기군" not in subsample["시군구"].to_list():
        print(3)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "충남", "시군구": "연기군"}])
        ])
    if "청원군" not in subsample["시군구"].to_list():
        print(4)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "충북", "시군구": "청원군"}])
        ])
    if "여주군" not in subsample["시군구"].to_list():
        print(5)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "경기", "시군구": "여주군"}])
        ])
    if "여주시" not in subsample["시군구"].to_list():
        print(6)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "경기", "시군구": "여주시"}])
        ])
    if "당진군" not in subsample["시군구"].to_list():
        print(7)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "충남", "시군구": "당진군"}])
        ])
    if "당진시" not in subsample["시군구"].to_list():
        print(8)
        new_rows = pd.concat([
            new_rows, pd.DataFrame([{"미분양": 0, "월": year, "구분": "충남", "시군구": "당진시"}])
        ])

    # 데이터 병합
    subsample = pd.concat([subsample, new_rows])
    subsample_index = subsample.set_index(keys="시군구")
    print(len(subsample_index),len(custom_order))
    print(subsample_index.tail(3))
    subsample_sorted = subsample_index.reindex(custom_order)
    result_form = pd.concat([result_form, subsample_sorted])

0
6
8
179 179
         level_0  index     월  구분 미분양
시군구                                  
세종특별자치시      NaN    NaN  2008  세종   0
여주시          NaN    NaN  2008  경기   0
당진시          NaN    NaN  2008  충남   0
0
6
8
179 179
         level_0  index     월  구분 미분양
시군구                                  
세종특별자치시      NaN    NaN  2009  세종   0
여주시          NaN    NaN  2009  경기   0
당진시          NaN    NaN  2009  충남   0
0
6
8
179 179
         level_0  index     월  구분 미분양
시군구                                  
세종특별자치시      NaN    NaN  2010  세종   0
여주시          NaN    NaN  2010  경기   0
당진시          NaN    NaN  2010  충남   0
0
1
2
6
8
179 179
     level_0  index     월  구분 미분양
시군구                              
진해시      NaN    NaN  2011  경남   0
여주시      NaN    NaN  2011  경기   0
당진시      NaN    NaN  2011  충남   0
1
2
6
7
179 179
     level_0  index     월  구분 미분양
시군구                              
진해시      NaN    NaN  2012  경남   0
여주시      NaN    NaN  2012  경기   0
당진군      NaN    NaN  2012  충남   0
1
2
6
7
179 17

In [48]:
# test = pd.read_excel("EA_Data/nonhouse2.0.xlsx")
subsample_index.iloc[0:20,:]
# subsample_sorted

,level_0,index,월,구분,미분양
시군구,,,,,
서울특별시,2411.0,2421.0,2022-12,서울,953
부산광역시,2412.0,2422.0,2022-12,부산,"2,640"
대구광역시,2413.0,2423.0,2022-12,대구,"13,445"
인천광역시,2414.0,2424.0,2022-12,인천,"2,494"
광주광역시,2415.0,2425.0,2022-12,광주,291
대전광역시,2416.0,2426.0,2022-12,대전,"3,239"
울산광역시,2417.0,2427.0,2022-12,울산,"3,570"
경기도,2418.0,2428.0,2022-12,경기,"7,588"
수원시,2419.0,2429.0,2022-12,경기,70


In [49]:
result_form.to_excel("nonhouse2.0.xlsx")